#Group SOMETHING 1D ML project
1. Loy Pek Yong 1004475


#Part 1

In [1]:
# import libraries
import pandas as pd
import numpy as np

# read training data


In [ ]:
"""Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation):
"""
def emission_param(training_set):
# Count the number of occurrences of each label and each (label, observed value) pair
    label_counts, emission_counts = {}, {}
    
    for observation, label in training_set:
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
        if (label, observation) not in emission_counts:
            emission_counts[(label, observation)] = 0
        emission_counts[(label, observation)] += 1

# Estimate the emission parameters from the counts
    emission_param = {}
    for label, observation in emission_counts:
        emission_param[(label, observation)] = emission_counts[(label, observation)] / label_counts[label]
    return emission_param
